In [13]:
import numpy as np
import pandas as pd
import os

#Set number of Monte Carlo Simulations
iterations = 100000

#Set Discount Rate Parameters
rf = 0.0192
erp = 0.0687
beta = 1.49
companyspread = 0.0125
countryspread = 0.009
taxrate = 0.19
alpha = 0.01
debtweight = 0.2748
terminal_growth = 0.01
debt_outstanding = 200000000
Cash_and_non_operating_assets = 15000000
Percent_equity_held = 0.75

#Set the filepath where the DCF data is stored
filepath = "D:\Investing Papers\Machine Learning"


# This code sets the current directory to the file path - DO NOT CHANGE
os.chdir(filepath)
cwd = os.getcwd()

#add the name of the excel file in the text in red, after the "r"
df = pd.read_excel(r'DCFTestData.xlsx', index_col=0,header=0)


#Calculate Cash Flows from input data

df['EBITDA'] = df['Revenue'] - df['Operating Cost']
df['EBIT'] = df['EBITDA'] - df['DA']
df['FCF'] = df['EBIT'] - df['Tax'] + df['DA'] - df['Capex'] - df['Change in WC']

#Simulate beta and company spread variables
beta_dist = np.random.normal(loc=beta, scale=0.01, size=iterations)
companyspread_dist = np.random.triangular(left=companyspread*0.5,mode=companyspread, 
                                          right=companyspread*1.5, size=iterations)

#Calculate WACCs
cost_of_debt = ((rf + companyspread_dist + countryspread) * (1-taxrate))
cost_of_equity = (rf + (beta_dist*erp) + alpha)
cost_of_capital = ((cost_of_debt*debtweight) + (cost_of_equity*(1-debtweight)))

#Convert input table into numpy array
df = df.to_numpy()

#Extract the discount rate periods
DisPeriods = df[:,6]

#Calculate Discount Rates
Discount_Rates = np.zeros((iterations,DisPeriods.shape[0]))
for i in range(0, DisPeriods.shape[0]):
    Discount_Rates[:,i] = 1/((1+cost_of_capital)**DisPeriods[i])


#Extract Cash Flows from numpy array and repeat cash flows for the number of Monte Carlo
#simulations to be performed

FCF = df[:,-1]
FCF = np.repeat(FCF[np.newaxis,:], iterations, 0)

#Calculate Terminal Value from the Terminal Years FCFF and reattach it to the FCF array
terminal_value = np.array(FCF[:,-1] *(1+terminal_growth)/(cost_of_capital-terminal_growth))
FCF = np.array(FCF[:,:-1])
FCF = np.column_stack((FCF,terminal_value))

#Simulate variance in FCF values and repeat/transpose to make 
FCF_dist_original = np.random.uniform(low=.90,high=1.1, size=iterations)
FCF_dist = np.transpose(np.repeat(FCF_dist_original[np.newaxis,:], np.shape(FCF)[1],0))

#Multiply Cash Flows by the simulated variance
FCFOutput = FCF*FCF_dist

#Multiply the FCF simulated values by the Discount Rate simulated values
DCF = FCFOutput * Discount_Rates

#Total Firm Simulation Value
DCFSum = np.sum(DCF,axis=1)

#Equity Simulation Value
EquityValue = (DCFSum-debt_outstanding+Cash_and_non_operating_assets)*Percent_equity_held

#Firm Value Percentiles
DCFpercentiles = np.array([np.percentile(DCFSum, 75),np.percentile(DCFSum, 50),np.percentile(DCFSum, 25),
                      np.percentile(DCFSum, 10),np.percentile(DCFSum, 1),np.percentile(DCFSum, 0.01)])
labels = np.array(["75th Percentile","50th Percentile","25th Percentile","10th Percentile",
                   "1st Percentile","0.01 Percentile"])

DCFpercentiles = np.column_stack((labels,DCFpercentiles))
DCFpercentiles = pd.DataFrame(DCFpercentiles)
DCFpercentiles.columns = ['Percentile','DCF Firm Value']


#Equity Value Percentiles, accounting for % of equity held
Equitypercentiles = np.array([np.percentile(EquityValue, 75),np.percentile(EquityValue, 50),np.percentile(EquityValue, 25),
                      np.percentile(EquityValue, 10),np.percentile(EquityValue, 1),np.percentile(EquityValue, 0.01)])

Equitypercentiles = np.column_stack((labels,Equitypercentiles))
Equitypercentiles = pd.DataFrame(Equitypercentiles)
Equitypercentiles.columns = ['Percentile','DCF Equity Value']

In [14]:
###Using the numexpr evaluate function instead of simple numpy

import numpy as np
import pandas as pd
import os
import numexpr as ne
import numba as nb

#Set number of threads for calculations
ne.set_num_threads(4)
def DCF():
    #Set number of Monte Carlo Simulations
    iterations = 100000

    #Set Discount Rate Parameters
    rf = 0.0192
    erp = 0.0687
    beta = 1.49
    companyspread = 0.0125
    countryspread = 0.009
    taxrate = 0.19
    alpha = 0.01
    debtweight = 0.2748
    terminal_growth = 0.01
    debt_outstanding = 200000000
    Cash_and_non_operating_assets = 15000000
    Percent_equity_held = 0.75

    #Set the filepath where the DCF data is stored
    filepath = "D:\Investing Papers\Machine Learning"


    #This code sets the current directory to the file path - DO NOT CHANGE
    os.chdir(filepath)
    cwd = os.getcwd()

    #add the name of the excel file in the text in red, after the "r"
    df = pd.read_excel(r'DCFTestData.xlsx', index_col=0,header=0)

    #Calculate Cash Flows from input data

    df['EBITDA'] = df['Revenue'] - df['Operating Cost']
    df['EBIT'] = df['EBITDA'] - df['DA']
    df['FCF'] = df['EBIT'] - df['Tax'] + df['DA'] - df['Capex'] - df['Change in WC']

    #Simulate beta and company spread variables
    beta_dist = np.random.normal(loc=beta, scale=0.01, size=iterations)
    companyspread_dist = np.random.triangular(left=companyspread*0.5,mode=companyspread, 
                                              right=companyspread*1.5, size=iterations)

    #Calculate WACCs
    ex ='((rf + companyspread_dist + countryspread) * (1-taxrate))'
    cost_of_debt = ne.evaluate(ex)

    ex = '(rf + (beta_dist*erp) + alpha)'
    cost_of_equity = ne.evaluate(ex)

    ex = '((cost_of_debt*debtweight) + (cost_of_equity*(1-debtweight)))'
    cost_of_capital = ne.evaluate(ex)

    #Convert input table into numpy array
    df = df.to_numpy()

    #Extract the discount rate periods
    DisPeriods = df[:,6]

    #Calculate Discount Rates
    Discount_Rates = np.zeros((iterations,DisPeriods.shape[0]))
    for i in range(0, DisPeriods.shape[0]):
        Discount_Rates[:,i] = 1/((1+cost_of_capital)**DisPeriods[i])


    #Extract Cash Flows from numpy array and repeat cash flows for the number of Monte Carlo
    #simulations to be performed

    FCF = df[:,-1]
    FCF = np.repeat(FCF[np.newaxis,:], iterations, 0)

    #Calculate Terminal Value from the Terminal Years FCFF and reattach it to the FCF array
    ex = 'np.array(FCF[:,-1] *(1+terminal_growth)/(cost_of_capital-terminal_growth))'
    terminal_value = terminal_value = ne.evaluate(ex)
    FCF = np.array(FCF[:,:-1])
    FCF = np.column_stack((FCF,terminal_value))

    #Simulate variance in FCF values and repeat/transpose to make 
    FCF_dist_original = np.random.uniform(low=.90,high=1.1, size=iterations)
    FCF_dist = np.transpose(np.repeat(FCF_dist_original[np.newaxis,:], np.shape(FCF)[1],0))

    #Multiply Cash Flows by the simulated variance
    ex = 'FCF*FCF_dist'
    FCFOutput = ne.evaluate(ex)

    #Multiply the FCF simulated values by the Discount Rate simulated values
    ex = 'FCFOutput * Discount_Rates'
    DCF = ne.evaluate(ex)

    #Total Firm Simulation Value
    DCFSum = np.sum(DCF,axis=1)

    #Equity Simulation Value
    ex = '(DCFSum-debt_outstanding+Cash_and_non_operating_assets)*Percent_equity_held'
    EquityValue = ne.evaluate(ex)
    
    return DCFSum, EquityValue


DCFSum, EquityValue = DCF()

#Firm Value Percentiles
DCFpercentiles = np.array([np.percentile(DCFSum, 75),np.percentile(DCFSum, 50),np.percentile(DCFSum, 25),
                      np.percentile(DCFSum, 10),np.percentile(DCFSum, 1),np.percentile(DCFSum, 0.01)])
labels = np.array(["75th Percentile","50th Percentile","25th Percentile","10th Percentile",
                   "1st Percentile","0.01 Percentile"])

DCFpercentiles = np.column_stack((labels,DCFpercentiles))
DCFpercentiles = pd.DataFrame(DCFpercentiles)
DCFpercentiles.columns = ['Percentile','DCF Firm Value']


#Equity Value Percentiles, accounting for % of equity held
Equitypercentiles = np.array([np.percentile(EquityValue, 75),np.percentile(EquityValue, 50),np.percentile(EquityValue, 25),
                      np.percentile(EquityValue, 10),np.percentile(EquityValue, 1),np.percentile(EquityValue, 0.01)])

Equitypercentiles = np.column_stack((labels,Equitypercentiles))
Equitypercentiles = pd.DataFrame(Equitypercentiles)
Equitypercentiles.columns = ['Percentile','DCF Equity Value']

In [10]:
import pandas as pd
import pandas_profiling

# Change file path and print for confirmation
import os
os.chdir("X:\ML")
cwd = os.getcwd()
print(cwd)

#open the data file
df = pd.read_csv(r'Melbourne_housing_FULL.csv')

#To display the report in the notebook - note this makes the code file large in terms of MB
#df.profile_report()

#To export the report to an HTML file usable outside of the notebook
profile = df.profile_report(title='Melbourn Housing Profile')
profile.to_file("Melbourne Housing Profile.html")


X:\ML


In [ ]:
import numba as nb

